In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [102]:
data_enero = pd.read_csv('./datasets/147_enero.csv', sep=';')
data_agosto = pd.read_csv('./datasets/147_agosto.csv', sep=';')
data_junio = pd.read_csv('./datasets/147_junio.csv', sep=';')
data = pd.concat([data_enero, data_agosto, data_junio], ignore_index=True)
poblaciones = pd.read_csv('caba_pob_barrios_2010.csv')


In [103]:
data.categoria.value_counts()

LIMPIEZA Y RECOLECCIÓN                   11648
TRÁNSITO                                  8280
CALLES Y VEREDAS                          2687
BARRIOS EMERGENTES                        2441
ARBOLADO Y ESPACIOS VERDES                2287
TRÁMITES Y SERVICIOS                      1258
ALUMBRADO                                  830
ORDENAMIENTO DEL ESPACIO PÚBLICO           781
RECICLADO Y PROTECCIÓN AMBIENTAL           709
CONTROL EDILICIO, OBRAS Y CATASTRO         495
FISCALIZACIÓN ACTIVIDADES COMERCIALES      493
MEDIOS DE TRANSPORTE                       464
PLUVIALES                                  120
SEGURIDAD                                  116
SALUD Y SERVICIOS SOCIALES                 115
DESARROLLO URBANO                           26
TERRENO BALDÍO                              21
LOTBA                                        2
EDUCACION                                    1
Name: categoria, dtype: int64

In [104]:
data = data[data['domicilio_barrio'].isin([
    'PALERMO', 'BARRACAS', 'RECOLETA', 'BELGRANO', 'CABALLITO', 'FLORES', 'ALMAGRO', 'NUÑEZ', 'PUERTO MADERO'])]


In [105]:
data = data[data['categoria'].isin(['TRÁNSITO', 'LIMPIEZA Y RECOLECCIÓN', 'CALLES Y VEREDAS', 'ORDENAMIENTO DEL ESPACIO PÚBLICO','SEGURIDAD'])]

In [106]:
data.shape

(8274, 17)

In [107]:
poblaciones = poblaciones[poblaciones['BARRIO'].isin([
    'PALERMO', 'BARRACAS', 'RECOLETA', 'BELGRANO', 'CABALLITO', 'FLORES', 'ALMAGRO', 'NUÑEZ', 'PUERTO MADERO'])]


In [108]:
data.domicilio_barrio.value_counts()
data_counts = data.domicilio_barrio.value_counts()


In [109]:
data_counts = data_counts.sort_index()



In [110]:
#get only the series values
data_counts = data_counts.values


In [111]:
poblaciones['CANTIDAD_RECLAMOS'] = data_counts

In [113]:
poblaciones['PUNTAJE'] = poblaciones['CANTIDAD_RECLAMOS'] / poblaciones['POBLACION'] * 100

In [116]:
poblaciones =poblaciones.reset_index(drop = True)

In [118]:
poblaciones.to_csv('147_merge_denuncias.csv', sep=';', index=False)